In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
training_dir='/kaggle/input/siim-isic-melanoma-classification/train.csv'
test_dir='/kaggle/input/siim-isic-melanoma-classification/test.csv'
train_dataframe=pd.read_csv(training_dir)
test_dataframe=pd.read_csv(test_dir)
train_dataframe.head(7)


In [ ]:
test=pd.read_csv(test_dir)

In [ ]:
train_dataframe['target'].value_counts()

In [ ]:
#percentage of people not suffering from Melanoma are:
print('Non-Melonoma patients are :',(32542/(32542+584)*100))
print('Melonoma patients are : ',(584/(32542+584)*100))

As we saw above we have 33126 training_dataset and in which there is only 584 having label 1 or suffered from Melanoma. 
So,here "data imblance" problem arises.
only 1.7% of people suffered from meloma in the given dataset

In [ ]:
#Format in which we need to submit the submission file.
submission=pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/sample_submission.csv')
submission.head(10)

In [ ]:
train_dataframe['sex'].value_counts() 
#It shows,there is no gender biasness in this dataset,both are approximately equal

In [ ]:
train_dataframe.describe()

***Mean of age is approx 49,and 75% of dataset lies under 60 ,
and 1remaining upto 90 so there are not much more outliers in the dataset w.r.t age***

In [ ]:
#'anatom_site_general_challenge' is the location of cancer,so it is very important feature.

labels=train_dataframe['anatom_site_general_challenge'].value_counts()
values=train_dataframe['anatom_site_general_challenge'].value_counts().values
# i will try to plot it using graph,i think pie chart is good 
fig=labels.plot.pie(y=values,figsize=(10,10),autopct='%1.1f%%',startangle=15, shadow = True)


**If we look at the graph carefully,
more than 51% of data lies under "torso"
it means half of the person having cancer on "torso"**

In [ ]:
train_dataframe.drop(train_dataframe.loc[train_dataframe['diagnosis']=='unknown'].index, inplace=True)

In [ ]:
#Another feature is diagnosis
#I try to remove the "unknown" valued rows.

diag_index=train_dataframe['diagnosis'].value_counts()
diag_labels=train_dataframe['diagnosis'].value_counts().values
diag_index,diag_labels

In [ ]:
diag_index.plot.pie(y=diag_labels,subplots=True,figsize=(7,5), startangle=15)

** 82% data are unknown  and 16% are navus**

In [ ]:
df=train_dataframe.drop(['image_name','patient_id','sex','age_approx','anatom_site_general_challenge','target'],axis=1)

In [ ]:
df

In [ ]:
# if we will try to figure out that in which category of diagnosis ,most of the people are maligant or not maligant
pd.crosstab(df['diagnosis'].values,df['benign_malignant'])

**benign cases are diagnosed as nevus
**and those diagnosed as "melanoma" are maligant**

In [ ]:
benign_data=train_dataframe[train_dataframe['target']==0].sample(1500) #i have taken a small sample of benign_data
maligant_data=train_dataframe[train_dataframe['target']==1]
train=pd.concat([benign_data,maligant_data])
train=train.reset_index()

In [ ]:
train

In [ ]:
train_dir='/kaggle/input/siim-isic-melanoma-classification/jpeg/train/'
test_dir='/kaggle/input/siim-isic-melanoma-classification/jpeg/test/'
data=[]
labels=[]
for i in range(train.shape[0]):
    data.append(train_dir+train['image_name'].iloc[i]+'.jpg')
    labels.append(train['target'].iloc[i])
df=pd.DataFrame(data)
df.columns=['images']
df['target']=labels

In [ ]:
train.shape,df.shape

In [ ]:
# ../input/siim-isic-melanoma-classification/jpeg/train

In [ ]:
test_data=[]
for i in range(test.shape[0]):
    test_data.append(test_dir + test['image_name'].iloc[i]+'.jpg')
df_test=pd.DataFrame(test_data)
df_test.columns=['images']

In [ ]:
df_test

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(df['images'],df['target'],test_size=0.2,random_state=42)

train_gen = pd.DataFrame({'image_dir': x_train, 'target': y_train})
val_gen = pd.DataFrame({'image_dir': x_val, 'target': y_val})

In [ ]:
train_gen

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,horizontal_flip=True)
val_datagen=ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
    train_gen,
    x_col='image_dir',
    y_col='target',
    target_size=(256, 256),
    batch_size=8,
    shuffle=True,
    class_mode='raw')

validation_generator = val_datagen.flow_from_dataframe(
    val_gen,
    x_col='image_dir',
    y_col='target',
    target_size=(256, 256),
    shuffle=False,
    batch_size=8,
    class_mode='raw')

In [ ]:
train_gen.shape,val_gen.shape

In [ ]:
#from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras import layers
from keras.metrics import AUC
from keras.activations import sigmoid
from keras.optimizers import SGD, Adam, Adamax
from keras import Model

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
model=EfficientNetB0(weights='imagenet',include_top=False,drop_connect_rate=0.4,input_shape=(256, 256, 3))


In [ ]:
x = layers.Flatten()(model.output)
output = layers.Dense(1, activation='sigmoid')(x)
model = Model(model.input, output)

In [ ]:
pip install focal-loss

In [ ]:
import keras
from focal_loss import BinaryFocalLoss
from keras.metrics import AUC
opt = keras.optimizers.Adam(lr = 1e-5)
# model.compile(loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'],optimizer=opt)
model.compile(loss=BinaryFocalLoss(gamma=2), metrics=['AUC'],optimizer=opt)

In [ ]:
nb_epochs = 25
batch_size=8
nb_train_steps = train_gen.shape[0]//batch_size
nb_val_steps=val_gen.shape[0]//batch_size
print("Number of training and validation steps: {} and {}".format(nb_train_steps,nb_val_steps))

In [ ]:
#CallBacks Function
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
early_stop=EarlyStopping(monitor="val_loss",
                         patience=10,
                         mode="auto",)
Learning_rate_reduction=ReduceLROnPlateau(monitor='val_loss',patience=2,verbose=1,factor=0.5,min_lr=0.001)

callbacks=[early_stop,Learning_rate_reduction]

In [ ]:
hist=model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_steps,
    epochs=nb_epochs,
    validation_data=validation_generator,
    callbacks=callbacks,
    validation_steps=nb_val_steps)

# history = model.fit_generator(train_generator,validation_data = validation_generator,epochs = 20, verbose = 1,callbacks=callbacks)

In [ ]:
import cv2
target=[]
for path in df_test['images']:
    img=cv2.imread(str(path))
    img = cv2.resize(img, (256,256))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    img=np.reshape(img,(1,256,256,3))
    prediction=model.predict(img)
    target.append(prediction[0][0])

submission['target']=target

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

acc = hist.history['auc']
val_acc = hist.history['val_auc']
loss = hist.history['loss']
val_loss = hist.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='green', label='Training Accuracy')
plt.plot(epochs, val_acc, color='blue', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
import pickle
Pkl_Filename = "Pickle_RL_Model.pkl"
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)